<a href="https://colab.research.google.com/github/Olhaau/fl-official-statistics-addon/blob/main/_dev/03_JW_tuned_insurance_federated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical Insurance - a Federated Learning Use Case.

This notebook contains Federated Learning.

-- **more tba** --

## Setup
---

In [10]:
# Is a repo-clone and installs needed (e.g. in colabs)? 
need_clone_install = False

### Pull Repo

In [11]:
if need_clone_install:
    import os
    
    # rm repo from gdrive
    if os.path.exists("fl-official-statistics-addon"):
      %rm -r fl-official-statistics-addon

    # clone
    !git clone https://github.com/Olhaau/fl-official-statistics-addon
    %cd fl-official-statistics-addon

    # pull (the currenct version of the repo)
    !git pull

### Installs

In [3]:
if need_clone_install: 
  !pip install --quiet nest_asyncio
  !pip install --quiet tensorflow_federated
  !pip install --quiet tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.0/349.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.6/532.6 kB 37.8 MB/s eta 0:

In [24]:
!python --version

Python 3.9.16


In [25]:
!pip list

Package                       Version
----------------------------- ------------
absl-py                       1.0.0
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
anyio                         3.6.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.15.1
astropy                       5.2.2
astunparse                    1.6.3
attrs                         21.4.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
blosc2                        2.0.0
bokeh                         2.4.3
branca                        0.6.0
CacheControl                  0.12.11
cached-property               1.5.2
cachetools                    3.1.1
catalogue   

### Imports

In [1]:
import collections
import numpy as np
import pandas as pd
import tqdm
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import tensorflow_federated as tff
#from tensorflow_addons.metrics import RSquare
#import tensorflow_addons as tfa
import nest_asyncio
from sklearn.metrics import r2_score

np.random.seed(0)

2023-05-09 20:44:26.681653: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 20:44:28.136565: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Ingest and Split the Data

In [2]:
df = pd.read_csv("../../output/data/insurance-clean.csv", index_col = 0)
df.head()

,age,sex,bmi,children,smoker,region,charges,region0,region1,region2,region3
0,0.021739,0.0,0.321227,0.0,1.0,southwest,16884.92400,0.0,0.0,0.0,1.0
1,0.000000,1.0,0.479150,0.2,0.0,southeast,1725.55230,0.0,0.0,1.0,0.0
2,0.217391,1.0,0.458434,0.6,0.0,southeast,4449.46200,0.0,0.0,1.0,0.0
3,0.326087,1.0,0.181464,0.0,0.0,northwest,21984.47061,0.0,1.0,0.0,0.0
4,0.304348,1.0,0.347592,0.0,0.0,northwest,3866.85520,0.0,1.0,0.0,0.0


In [3]:
NUM_CLIENTS = 4
NUM_EPOCHS = 50
BATCH_SIZE = 128
SHUFFLE_BUFFER = 20
PREFETCH_BUFFER = 5
NUM_ROUNDS = 50
RUN_NAME = f'0,8-3({NUM_ROUNDS})-{NUM_EPOCHS}-epochs-{BATCH_SIZE}-batch-WithRegion/'

In [50]:
syn_samples_per_region = 1000

def get_dataset_for_region(dataset, region_index, test_size_per_region=20):
    """Min-max scale and return data for a single, given region. The scaler must be fitted before.

    :param dataset: The dataset to get the regional data from
    :type dataset: pandas.DataFrame
    :param region_index: The index number of the region to return
    :type region_index: int
    :param test_size_per_region: The amount of values to separate for testing, default are 20
    :type test_size_per_region: int, optional
    :return: The dataset specific for the defined region, the test values, the test labels
    :rtype: tensorflow.python.data.ops.dataset_ops.PrefetchDataset, tuple of pandas.core.series.Series
    """
    region_ds = dataset[dataset['region'] == region_index]
    region_ds = region_ds.drop(columns=['region'])
    len = region_ds.shape[0]

    # The scaling into [0, 1] is not necessary anymore, it happens when the data loads already
    # region_ds[['age', 'bmi', 'children']] = scaler.transform(region_ds[['age', 'bmi', 'children']])

    X_test = region_ds.head(test_size_per_region)
    y_test = X_test.pop('charges')

    X_train = region_ds.tail(len - test_size_per_region)
    y_train = X_train.pop('charges')

    fed_train_dataset = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)))

    return (
        fed_train_dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER),
        (X_test, y_test)
    )

In [51]:
# Create test and train sets and put them into random_client_ds, use four clients which are independent of the region
def get_dataset_random_region(dataset, num_clients=4, test_size_per_region=20):
    """Creates a list with client datasets independent of the region.

    :param dataset: The dataset to get the regional data from
    :type dataset: pandas.DataFrame
    :param num_clients: the number of clients create (equal big datasets per client), default value is 4 clients
    :type num_clients: int, optional
    :param test_size_per_region: The amount of values to separate for testing, default are 20
    :type test_size_per_region: int, optional
    :return: List of the prepared dataset with one entry per region, the test values and labels for each region
    :rtype: List of (tensorflow.python.data.ops.dataset_ops.PrefetchDataset, tuple of pandas.core.series.Series)"""
    size_of_client_ds = int(dataset.shape[0] / num_clients)

    dataset_to_split = dataset.copy()
    dataset_to_split.pop("region")
    random_client_ds = []
    for i in range(num_clients):
        sampled = dataset_to_split.sample(n=size_of_client_ds)
        dataset_to_split.drop(sampled.index)

        X_test = sampled.head(test_size_per_region)
        y_test = X_test.pop('charges')

        X_train = sampled.tail(size_of_client_ds - test_size_per_region)
        print(len(X_train))
        y_train = X_train.pop('charges')

        fed_train_dataset = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)))

        train_set = fed_train_dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER)
        print(train_set)
        test_set = (X_test, y_test)

        random_client_ds.append((train_set, test_set))

    return random_client_ds

In [52]:
# Training data for clients with regional data (each client one region)
test_size_per_region = 20
regions = ['region0', 'region1', 'region2', 'region3']

federated_insurance_data = [
    get_dataset_for_region(df.drop(regions, axis=1), i, test_size_per_region=test_size_per_region)
    for i in range(NUM_CLIENTS-1)]
federated_insurance_data

[(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 5), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
  (Empty DataFrame
   Columns: [age, sex, bmi, children, smoker]
   Index: [],
   Series([], Name: charges, dtype: float64))),
 (<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 5), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
  (Empty DataFrame
   Columns: [age, sex, bmi, children, smoker]
   Index: [],
   Series([], Name: charges, dtype: float64))),
 (<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 5), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
  (Empty DataFrame
   Columns: [age, sex, bmi, children, smoker]
   Index: [],
   Series([], Name: charges, dtype: float64)))]

In [53]:
# Training data for clients with regional independent data
random_client_ds = get_dataset_random_region(df, num_clients=NUM_CLIENTS, test_size_per_region=test_size_per_region)

314
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 9), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>
314
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 9), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>
314
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 9), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>
314
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 9), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>


In [54]:
len(list(random_client_ds[0][0].as_numpy_iterator())[0])

2023-05-09 21:05:26.723141: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [314]
	 [[{{node Placeholder/_1}}]]


2

In [55]:

# show random clients
i = 0 
for client_dfs in random_client_ds:
    print('================================ client {} =================================='.format(i))
    client_train = list(client_dfs[0].as_numpy_iterator())
    client_test = list(client_dfs[1])
    print('--- Train (obs = {}) ------------------------------------------------------'.format(len(X_train)))
    print(pd.DataFrame(client_train[:3], columns = ["X", "y"]))
    print('-- Test (obs = {}) ---------------------------------------------------------'.format(len(client_test[0])))
    #print('obs: {}')
    print([x[:3] for x in client_test])
    i += 1

================================ client 0 ==================================
--- Train (obs = 1338) ------------------------------------------------------
                                                   X  \
0  [[0.0217391304347826, 0.0, 0.3346785041700295,...   
1  [[0.8478260869565217, 1.0, 0.208232445520581, ...   
2  [[0.3478260869565217, 1.0, 0.4530535377993003,...   

                                                   y  
0  [2331.519, 11520.09985, 9182.17, 3213.62205, 3...  
1  [10959.33, 1263.249, 2689.4954, 9875.6804, 171...  
2  [14358.36437, 26125.67477, 5458.04645, 2150.46...  
-- Test (obs = 20) ---------------------------------------------------------
[          age  sex       bmi  children  smoker  region0  region1  region2  \
550  0.978261  1.0  0.399247       0.0     0.0      0.0      0.0      0.0   
23   0.347826  0.0  0.429379       0.2     1.0      1.0      0.0      0.0   
841  0.891304  1.0  0.235136       0.0     0.0      1.0      0.0      0.0   

     region3 

2023-05-09 21:05:27.046086: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [314,9]
	 [[{{node Placeholder/_0}}]]
2023-05-09 21:05:27.189868: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [314]
	 [[{{node Placeholder/_1}}]]


--- Train (obs = 1338) ------------------------------------------------------
                                                   X  \
0  [[0.8260869565217392, 0.0, 0.2512779122948614,...   
1  [[0.6304347826086956, 1.0, 0.2542372881355932,...   
2  [[1.0, 0.0, 0.4574926015603981, 0.0, 0.0, 0.0,...   

                                                   y  
0  [11070.535, 4719.52405, 48675.5177, 9386.1613,...  
1  [21978.6769, 13822.803, 8413.46305, 21595.3822...  
2  [14692.66935, 46113.511, 9048.0273, 4266.1658,...  
-- Test (obs = 20) ---------------------------------------------------------
[          age  sex       bmi  children  smoker  region0  region1  region2  \
421  0.934783  1.0  0.535378       0.0     1.0      0.0      0.0      1.0   
298  0.282609  1.0  0.495830       0.6     1.0      0.0      1.0      0.0   
150  0.369565  1.0  0.219801       0.2     0.0      0.0      1.0      0.0   

     region3  
421      0.0  
298      0.0  
150      0.0  , 421    46599.1084
298    3874

2023-05-09 21:05:27.331597: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [314]
	 [[{{node Placeholder/_1}}]]


### Code Tests

In [56]:
print("splitted datasets: {}".format(len(federated_insurance_data)))
print("------------------------------")
# Q: why range(NUM_CLIENTS-1) and, thus, only 3 clients?

# Q: how to output? -> does not work or nothing in it?
# -> i think its empty
list(federated_insurance_data[0][0].as_numpy_iterator())[:10]

splitted datasets: 3
------------------------------


2023-05-09 21:05:27.528203: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [0]
	 [[{{node Placeholder/_1}}]]


[]

#### Show Tensor

In [57]:
# Create a Tensor Object
X_train = df.iloc[:,0:4]
y_train = df['charges']
df2 = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)))
df2

<_TensorSliceDataset element_spec=(TensorSpec(shape=(4,), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None))>

In [58]:
# Show Tensor Object
# S. https://stackoverflow.com/questions/62436302/extract-target-from-tensorflow-prefetchdataset
list(df2.as_numpy_iterator())[:10]

2023-05-09 21:05:27.794206: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [1338]
	 [[{{node Placeholder/_1}}]]


[(array([0.02173913, 0.        , 0.3212268 , 0.        ]), 16884.924),
 (array([0.        , 1.        , 0.47914985, 0.2       ]), 1725.5523),
 (array([0.2173913 , 1.        , 0.45843422, 0.6       ]), 4449.462),
 (array([0.32608696, 1.        , 0.18146355, 0.        ]), 21984.47061),
 (array([0.30434783, 1.        , 0.34759214, 0.        ]), 3866.8552),
 (array([0.2826087 , 0.        , 0.26311542, 0.        ]), 3756.6216),
 (array([0.60869565, 0.        , 0.47027172, 0.2       ]), 8240.5896),
 (array([0.41304348, 0.        , 0.31692225, 0.6       ]), 7281.5056),
 (array([0.41304348, 1.        , 0.37315039, 0.4       ]), 6406.4107),
 (array([0.91304348, 0.        , 0.26580576, 0.        ]), 28923.13692)]

## Model Builder

In [59]:
from keras.layers import ReLU
from keras.models import Sequential
from keras.layers import Dense

def create_keras_model(
  input_features = 9
  ):
  model = Sequential()
  model.add(Dense(40, input_dim=9))
  model.add(ReLU())
  model.add(Dense(40))
  model.add(ReLU())
  model.add(Dense(20))
  model.add(ReLU())
  model.add(Dense(1))
  return model


# A helper function for federated learning
def model_fn(    
  input_features = 9):

  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model(
          input_features = input_features
  )
  return tff.learning.models.from_keras_model(
      keras_model,
      # without region: 
      #input_spec = federated_insurance_data[0][0].element_spec,
      input_spec = random_client_ds[0][0].element_spec,
      loss = tf.keras.losses.MeanAbsoluteError(),
      #loss = tf.keras.losses.MeanSquaredError(),
      metrics = [tf.keras.metrics.MeanAbsoluteError()
      #metrics = [tf.keras.losses.MeanSquaredError()
      #,tfa.metrics.RSquare()
      ]
  )

# Helper functions for different features as input
def model_fn_5():
    return model_fn(5)

def model_fn_5_mod():
  return model_fn(5, initializer = 'glorot_uniform', activation = 'relu')

def model_fn_9():
  return model_fn(9)

def model_fn_9_mod():
  return model_fn(9, initializer = 'glorot_uniform', activation = 'relu')

## Federated Learning

### Tuned Model

#### Create Learning Process

In [60]:
# Create iterative learning process which will perform the federated learning
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn_9,
    client_optimizer_fn=lambda: tf.optimizers.Adam(learning_rate = .05),
    server_optimizer_fn=lambda: tf.optimizers.Adam(learning_rate = .05),)

KeyboardInterrupt: 

In [ ]:
# The initial setup of the learning process
print(iterative_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[9,40],
      float32[40],
      float32[40,40],
      float32[40],
      float32[40,20],
      float32[20],
      float32[20,1],
      float32[1]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64,
    float32[9,40],
    float32[9,40],
    float32[40],
    float32[40],
    float32[40,40],
    float32[40,40],
    float32[40],
    float32[40],
    float32[40,20],
    float32[40,20],
    float32[20],
    float32[20],
    float32[20,1],
    float32[20,1],
    float32[1],
    float32[1]
  >
>@SERVER)


#### Train

In [ ]:
state = iterative_process.initialize()

2023-05-09 20:45:31.870098: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-05-09 20:45:31.870370: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-09 20:45:31.947293: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-05-09 20:45:31.947623: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


In [ ]:
# Train the federated model with random clients
for round_num in tqdm.tqdm(range(1, 10)):
    result = iterative_process.next(state, [f[0] for f in random_client_ds])
    state = result.state
    metrics = result.metrics
    for name, value in metrics['client_work']['train'].items():
        tf.summary.scalar(name, value, step=round_num)

100%|██████████| 9/9 [00:03<00:00,  2.39it/s]


In [ ]:
result.metrics

OrderedDict([('distributor', ()),
             ('client_work',
              OrderedDict([('train',
                            OrderedDict([('mean_absolute_error', 2112.951),
                                         ('loss', 2114.2634),
                                         ('num_examples', 62800),
                                         ('num_batches', 492)]))])),
             ('aggregator',
              OrderedDict([('mean_value', ()), ('mean_weight', ())])),
             ('finalizer', OrderedDict([('update_non_finite', 0)]))])

In [ ]:
result.state
# all weights are zero
# -> nothing is learned 

LearningAlgorithmState(global_model_weights=ModelWeights(trainable=[array([[-1.56626120e-01,  1.55043912e+00, -5.04943252e-01,
        -1.15384471e+00,  1.26722324e+00,  1.40840268e+00,
         1.70245028e+00,  1.78481555e+00,  1.18387461e+00,
         1.73568487e+00,  2.01503202e-01,  8.49910557e-01,
        -6.58354402e-01, -5.44890940e-01,  1.00816357e+00,
         1.93432570e-02,  1.76440918e+00,  1.79244387e+00,
        -4.22498167e-01,  1.54786122e+00,  1.97519973e-01,
         1.56759059e+00,  1.69009256e+00,  4.69006717e-01,
         1.42993167e-01,  8.06179047e-01,  1.24127269e-01,
        -4.65176910e-01,  1.85895789e+00,  1.43365777e+00,
         1.80644488e+00,  2.33412087e-01,  1.83048189e+00,
         1.42468965e+00,  1.45335495e-03, -4.75021541e-01,
         1.34082401e+00,  5.92036173e-02,  5.75583614e-02,
         1.80231392e+00],
       [ 1.34063559e-03, -3.80669296e-01,  1.62117034e-01,
        -2.58858114e-01, -8.30026805e-01,  2.85833329e-01,
         1.07145049e-

#### Evaluate

In [ ]:
# Create the test data for model evaluation
X_test = pd.concat([f[1][0] for f in random_client_ds])
y_test = pd.concat([f[1][1] for f in random_client_ds])

test_sets = [tf.data.Dataset.from_tensor_slices(
    (tf.convert_to_tensor(np.expand_dims(el[1][0], axis=0)), 
    tf.convert_to_tensor(np.expand_dims(el[1][1], axis=0)))) 
    for el in random_client_ds]

In [ ]:
# Model evaluation
evaluation = tff.learning.build_federated_evaluation(model_fn_9)
# print(evaluation.type_signature.formatted_representation())
model_weights = iterative_process.get_model_weights(state)
train_metrics = evaluation(model_weights, test_sets)
train_metrics

/tmp/ipykernel_5164/771486142.py:2: DeprecationWarning: `tff.learning.build_federated_evaluation` is deprecated, use `tff.learning.algorithms.build_fed_eval` instead.
  evaluation = tff.learning.build_federated_evaluation(model_fn_9)
2023-05-09 20:47:07.430208: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-05-09 20:47:07.430461: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-09 20:47:07.543333: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-05-09 20:47:07.543661: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-09 20:47:07.562600: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-05-09 20:47:07.562910: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-09 20:47

OrderedDict([('eval',
              OrderedDict([('mean_absolute_error', 2001.5652),
                           ('loss', 2001.5652),
                           ('num_examples', 80),
                           ('num_batches', 4)]))])

In [ ]:
# Create model from training results and evaluate
model = create_keras_model(input_features = 9)
model_weights.assign_weights_to(model)
model.compile(
    loss=tf.losses.mae,
    # loss=tf.losses.mean_squared_error,
    optimizer=tf.optimizers.Adam(),
    metrics=["mae", 'mean_squared_error']
)
# The evaluation results, for technical reasons the metrics_names is called afterwards. However, its order fits to the results
print(model.evaluate(X_test, y_test))
print(model.metrics_names)

3/3 [==============================] - 0s 7ms/step - loss: 3185.3455 - mae: 3185.3455 - mean_squared_error: 42205208.0000
[3185.345458984375, 3185.345458984375, 42205208.0]
['loss', 'mae', 'mean_squared_error']


Results federated:

fl-loss (mae): 2280.2578 
🚀 JW

OH: 3185.3455